# Extract player info

In [28]:
from bs4 import BeautifulSoup
import urllib.request as url
import re

def find_player_id():
    players = {}
    player_pages = ["http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false", "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/41", "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/81", "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/121", "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/121", "http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/161","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/201","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/241","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/281","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/321","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/361","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/401","http://www.espn.com/nba/statistics/player/_/stat/scoring-per-game/sort/avgPoints/qualified/false/count/441"]
    for page in player_pages:
        html_page = url.urlopen(page)
        soup = BeautifulSoup(html_page)
        for link in soup.findAll('a', attrs={'href': re.compile("^http://www.espn.com/nba/player/_/id")}):
            players_id = link.get('href').split('/')
            players[players_id[len(players_id)-1]] = players_id[len(players_id)-2]
    return players;    

In [29]:
import pandas
def read_players(player_id, years):
    player_info = {}
    for player in player_id:
        player_years = {}
        for year in years:
            url_path = "http://www.espn.com/nba/player/gamelog/_/id/" + player_id[player] + "/year/" + str(year) + "/" + player
            try:
                dataframe = pandas.read_html(url_path)
                player_years[year] = dataframe
            except url.HTTPError:
                pass
            print(url_path)
        player_info[player] = player_years
    return player_info

In [30]:
player_id = find_player_id()
test = read_players(player_id,[2018])

KeyboardInterrupt: 

In [ ]:
train = read_players(player_id, [2013,2014,2015,2016,2017])

In [ ]:
table = train['jarell-martin'][2017]
print(table[1])

# Extract team info

In [31]:
def find_teams_url(team_url):
    html_page = url.urlopen(team_url)
    soup = BeautifulSoup(html_page)
    team_url = []
    for link in soup.findAll('a', attrs={'href': re.compile("^http://www.espn.com/nba/team/_/name/")}):
        team_url.append(link.get('href'))
    return set(team_url)


def read_teams(team_url, years):
    team_stat = {}
    for url_path in team_url:
        url_component = url_path.split('/_/')
        stat_url = url_component[0] + '/schedule/_/' + 'name/' + url_component[1].split('/')[1] + '/season/'
        year_stat = {}
        for year in years:
            try:
                dataframe = pandas.read_html(stat_url + str(year) + '/seasontype/2')
                year_stat[year] = dataframe
            except url.HTTPError:
                pass
            print(stat_url + str(year) + '/seasontype/2')
        frames = []
        for year in years:
            df_changed = year_stat[year][2].loc[2:, 0:2]
            before_half = df_changed[0].str.split(" ",expand=True)[1].isin(["Jan","Feb","Mar","Apr","May","Jun","Jul"])
            df_changed[0][before_half] = df_changed[0][before_half]+', '+str(year)
            df_changed[0][~before_half] = df_changed[0][~before_half]+', '+str(year-1)
            df_changed[3] = str(year - 1) + "-" + str(year)
            frames.append(df_changed)
        result = pandas.concat(frames)
        result.index = range(len(result))
        
        team_name_component = url_component[1][5:].split('/')[1].split('-')[:-1]
        team_name = ' '.join(team_name_component)
        
        team_stat[team_name] = result
    return team_stat

def read_teams_total(team_url, years):
    team_total = {}
    for url_path in team_url:
        url_component = url_path.split('/_/')
        stat_url = url_component[0] + '/stats/_/' + 'name/' + url_component[1].split('/')[1] + '/year/'
        year_total = {}
        for year in years:
            try:
                dataframe = pandas.read_html(stat_url + str(year))
                dataframe = pandas.concat([dataframe[0][len(dataframe[0])-1:],dataframe[1][len(dataframe[1])-1:]])
                dataframe.index = range(len(dataframe))
                year_total[year] = dataframe
            except url.HTTPError:
                pass
            print(stat_url + str(year))
        team_total[url_component[1][5:]] = year_total
    return team_total

In [32]:
team_url = find_teams_url("http://www.espn.com/nba/teams")
years = [2018]
test = read_teams(team_url, years)

http://www.espn.com/nba/team/schedule/_/name/gs/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/orl/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/chi/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/utah/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/okc/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/cle/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/no/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/lal/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/atl/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/det/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/hou/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/lac/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/den/season/2018/seasontype/2
http://www.espn.com/nba/team/schedule/_

In [33]:
print(test.keys())
# print(test['bos/boston-celtics'])
# from datetime import datetime
# datetime_object1 = datetime.strptime(test['bos/boston-celtics'][2018][2][0][2], "%a, %b %d")
# datetime_object2 = datetime.strptime(test['bos/boston-celtics'][2018][2][0][4], "%a, %b %d")
# (datetime_object2 - datetime_object1).days

dict_keys(['golden state', 'orlando', 'chicago', 'utah', 'oklahoma city', 'cleveland', 'new orleans', 'los angeles', 'atlanta', 'detroit', 'houston', 'la', 'denver', 'new york', 'minnesota', 'portland trail', 'memphis', 'toronto', 'washington', 'philadelphia', 'san antonio', 'charlotte', 'miami', 'brooklyn', 'dallas', 'indiana', 'sacramento', 'boston', 'milwaukee', 'phoenix'])


In [34]:
train = read_teams(team_url, [2013,2014,2015,2016,2017])

http://www.espn.com/nba/team/schedule/_/name/gs/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/gs/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/gs/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/gs/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/gs/season/2017/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/orl/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/orl/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/orl/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/orl/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/orl/season/2017/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/chi/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/chi/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/chi/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/nam

http://www.espn.com/nba/team/schedule/_/name/mia/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/mia/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/mia/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/mia/season/2017/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/bkn/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/bkn/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/bkn/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/bkn/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/bkn/season/2017/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/dal/season/2013/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/dal/season/2014/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/dal/season/2015/seasontype/2
http://www.espn.com/nba/team/schedule/_/name/dal/season/2016/seasontype/2
http://www.espn.com/nba/team/schedule/

KeyError: 'bos/boston-celtics'

In [36]:
from datetime import datetime
def data_processing(test):
    game_info = {}
    for team in test:
        for i in range(5,len(test[team][1])):
            #opponent of the current game
            if team not in game_info:
                game_info[team] = []
            #the results of former 5 games of the team
            info = [test[team][1][i][2:].strip().lower()]
            for j in range(1,6):
                if test[team][2][5-j][0] == 'L':
                    info.append('0')
                else:
                    info.append('1')
            #Home field or away of the current game
            if (test[team][1][i][1] == 'v'):
                info.append('1')
            else:
                info.append('0')
            #back to back
            d1 = datetime.strptime(test[team][0][i], "%a, %b %d, %Y")
            d2 = datetime.strptime(test[team][0][i-1], "%a, %b %d, %Y")
            if (d1-d2).days==1:
                info.append('1')
            else:
                info.append('0')
            #the feature vector and result of the current game       
            if (test[team][2][i][0] == 'L'):
                game_info[team].append((info, '0', test[team][3][i]))
            else:
                game_info[team].append((info, '1', test[team][3][i]))
                
    return game_info

In [ ]:
def total_to_dict(data):
    total_dict = {}
    for team in data:
        total_dict[team] = {}
        for year in data[team]:
            lst = [data[team][year][1][0]]
            for i in range(4, 14):
                lst.append(str(data[team][year][i][0]))
            for i in range(1, 15):
                lst.append(str(data[team][year][i][1]))
            total_dict[team][str(year-1) + '-' + str(year)] = lst
    return total_dict

teams_stat_test = read_teams_total(team_url, [2018])
teams_stat_train = read_teams_total(team_url, [2013,2014,2015,2016,2017])

teams_stat_test = total_to_dict(teams_stat_test)
teams_stat_train = total_to_dict(teams_stat_train)

In [ ]:
test_total

In [37]:
import pickle 

all_games_train = data_processing(train)
all_games_test = data_processing(test)

# with open('all_games_train.pkl', 'wb') as fp:
#     pickle.dump(all_games_train, fp)
# with open('all_games_test.pkl', 'wb') as fp:
#     pickle.dump(all_games_train, fp)

In [ ]:

import pickle 


# with open('teams_stat_train.pkl', 'wb') as fp:
#     pickle.dump(teams_stat_train, fp)
# with open('teams_stat_test.pkl', 'wb') as fp:
#     pickle.dump(teams_stat_test, fp)